In [1]:
from collections import deque

In [2]:
filename = "sample.txt"
# filename = "sample2.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

lines = data.strip().split("\n")

https://adventofcode.com/2024/day/12

In [3]:
## Part 1
# Map garden plots growing different plants
# Plants are part of the same plot if they have the same type and are adjacent
# Draw a fence around each plot. Price = Area x Perimeter
directions = [1, 1j, -1, -1j]  # ESWN
grid = {x + y * 1j: c for y, line in enumerate(lines) for x, c in enumerate(line)}

def adjacent(pos: complex) -> list[complex]:
    return [pos + step for step in directions]

def get_region(start_pos: complex) -> set[complex]:
    plant_type = grid[start_pos]
    frontier = deque([start_pos])
    seen = set()
    result = set()
    while frontier:
        pos = frontier.pop()
        # Have we seen this before?
        if pos in seen:
            continue
        seen.add(pos)
        # Is this the right type of plant?
        if grid[pos] != plant_type:
            continue

        result.add(pos)
        # Try neighbours of pos if they're on the map
        neighbours = adjacent(pos)
        frontier.extend(n for n in neighbours if n in grid)

    return result

def get_perimeter(region: set[complex]) -> int:
    # Note: The same outside-tile can be adjacent to more than 1 in the region. It should be counted multiple times
    perimeter = 0
    for pos in region:
        # Note: It doesn't matter if the neighbour isn't on the grid, it still counts!
        outside_neighbours = set(adjacent(pos)) - region
        perimeter += len(outside_neighbours)
    return perimeter

In [4]:
result = []
price = 0
seen = set()
for y, line in enumerate(lines):
    for x, c in enumerate(line):
        pos = x + y * 1j
        if pos in seen:
            continue
        region = get_region(pos)
        area = len(region)
        perimeter = get_perimeter(region)
        result.append((region, area))
        price += area * perimeter
        # Mark all positions covered by this region as seen
        seen.update(region)

print(f"{len(result)=}")
print(price)

len(result)=5
140


In [5]:
## Part 2
# Unfortunately they have a bulk discount
# For each region, Price = Area * n sides
# Note: count internal sides as well
# Note: Sides don't connect diagonally!

# Attempt 2:
# Build up a running count of new edges, plus a set of (pos, facing) that captures every edge
# for every cell, see if it's an edge
#   add it to the seen set
#   If the previous (pos, facing) for that edge hasn't been seen, unique_edges += 1
price = 0
for region, area in result:
    # Find the number of edges in this region
    edges = set()
    edge_count = 0
    # Go through the tiles in this region in (x, y) order
    for pos in sorted(region, key=lambda x: (x.imag, x.real)):
        for facing in directions:
            # Look ESWN. Is it an edge?
            if (pos + facing) in region:
                continue
            edges.add((pos, facing))
            # Does this follow on from an already-seen edge?
            # If facing N/S, prev=W
            # If facing E/W, prev=N
            match facing:
                case -1j | 1j:
                    prev_pos = pos - 1
                case -1 | 1:
                    prev_pos = pos - 1j
            if (prev_pos, facing) not in edges:
                # New edge!
                edge_count += 1
    price += area * edge_count

print(price)

80
